<a href="https://colab.research.google.com/github/rsmarinho/pdscodes/blob/master/TP05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# here goes your include modules...
import numpy as np
import matplotlib.pyplot as plt

# in this TP we will use scipy module to read the files
from scipy.io import wavfile
# if you rather to use librosa, use the lines below
import librosa

# below are the module used to play the audio vectors in jupyterlab
from IPython.display import Audio

**TP04 - Espectrograma.**

---

O espectrograma é uma representação visual da variação do espectro de frequência ao longo do tempo. Um formato comum do espectrograma é um gráfico de duas dimensões geométricas, um eixo é o eixo de representaçõa temporal e o outro é o de representação frequêncial.

O espectrograma pode ser gerado de várias maneiras, seja com um banco de filtros (*filterbank*) ou com a utilização da transformada de Fourier (DFT). Nesse exemplo utilzaremos a transformada rápida de Fourier (FFT) para criação do espectrograma.

Este trabalho prático será utilizado no intuito de estudar diferentes tipos de visualização de informação, bem como de aumentar sua proficiência na manipulação de sinais digitais.

In [ ]:
# O arquivo de audio utilizado está no github, antes de 
# fazer as células funcionarem faça o download do arquivo 
# caprice24mono.wav e coloque-o em uma pasta no drive 
# para que você possa utilizá-lo com o google colab.

# O arquivo caprice24mono.wav é um extrato do arquivo
# encontrado em https://en.wikipedia.org/wiki/File:Paganini_Caprice-24.ogg
# O intervalo do arquivo é entre os segundos 27 e 32, e
# foi escolhido pois é possível escutar com nitidez os dois
# instrumentos (violino e piano).

data_sr, data = wavfile.read('/content/drive/caprice24mono.wav')
Audio(data, rate=data_sr)

O espectograma de um sinal unidimensional é obtido com o cálculo da FFT de uma janela (intervalo do sinal). Essa janela "*anda*" ao longo do tempo e podem se sobrepor ou não. A movimentação dessa janela ao longo do tempo é definida por um *overlap*, e no caso do overlap ter o mesmo tamanho da janela, então as janelas não se sobrepõem.

As células abaixo contém uma função cada. A primeira calcula a fft de um intervalo do sinal, e a segunda retornam uma *matriz* com o espectograma calculado.

In [ ]:
# Calcula a FFT de um intervalo do sinal s.
# O intervalo é definido pelas variáveis a e b
# em uma versão mais completa (sem os comentários)
# a função retorna também os índices de frequência
# da fft

def getFFTFromWindow(s, a, b, fs=data_sr):
    sig = s[a:b]
    fftsig = np.fft.fft(sig)
    fftsig = np.fft.fftshift(fftsig)

    N = len(fftsig)
    # freq = np.fft.fftfreq(N, 1/fs)
    # freq = np.fft.fftshift(freq)

    # return freq[:int(N/2):], fftsig[:int(N/2)]
    return fftsig[:int(N/2)]

In [ ]:
# Retorna o espectrograma de um sinal data
# A variável fs é a frequência de amostragem do sinal
# data (não é utilizado no código). Nessa função é 
# importante definir o tamanho da janela (window) e 
# qual é a sobre posição das janelas (overlap) para 
# o cálculo da fft do intervalo do sinal

def spectrogram(data, fs, window=1000, overlap=500):
    total = len(data)

    spectrogram = np.empty((1, int(window/2)))
    
    i = 0
    while True:
      spec = getFFTFromWindow(data, i*overlap, i*overlap + window)

      if len(spec) != int(window/2):
        break
      spectrogram = np.vstack((spectrogram, np.abs(spec[:])))
      i+=1

    return np.absolute(spectrogram.T)

Uma das maneiras de visualizar e compreender o espectrograma é **"como o sinal varia em frequência ao longo do tempo"**, para isso a visualização em 3d da matriz do espectrograma é uma boa maneira de proceder.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Cálculo e visualização do espectrograma de 
# um sinal de voz (data), com janelamento de 1000
# amostras e overlap de 100 amostras.


# Cálculo do espectrograma
spectro = spectrogram(data, data_sr, window=1000, overlap=100)

# Visualização da matriz do espectrograma
# Set up grid and test data
ny = np.shape(spectro)[0]
nx = np.shape(spectro)[1]

x = range(nx)
y = range(ny)

hf = plt.figure()
ha = hf.add_subplot(111, projection='3d')

X, Y = np.meshgrid(x, y)
ha.plot_surface(X, Y, spectro)

plt.show()

De uma maneira mais simples, a matriz do espectrograma é *explicada* como uma matriz de intensidades (já que em nosso caso a função espectrograma retorna a magnitude do sinal). Daí a visualização em 2d.

Note que os atributos *vmin* e *vmax* são utilizados para melhor vizualização.

In [ ]:
plt.imshow(spectro/np.mean(spectro), aspect='auto', cmap='inferno', vmin=0, vmax=8)

Responda:

**1.** Explique com suas palavras como é extraído o espectrograma de um sinal.

**2.** Modifique a função spectrogram para que diferentes tipos de janelas possam ser utilizados na variável *data* no interior da função spectrogram.

**3.** Faça um espectrograma com utilizando um janelamento do tipo gaussiano e faça o plot de visualização utilizando
```
plt.imshow(spectro/np.mean(spectro), aspect='auto', cmap='inferno', vmin=0, vmax=8)
```

O que aconteceu? Agora utilize
```
plt.imshow(spectro, aspect='auto', cmap='inferno', vmin=0, vmax=1)
```

Explique.

**4.** Crie um vetor com a representação da função [chirp](https://en.wikipedia.org/wiki/Chirp) e visualize o espectrograma. Explique o que acontece.

**5.** Um sinal FM é um sinal que tem a frequência de uma sinal de frequência portadora modificado por um sinal de mensagem. De maneira carente de precisão, pode-se dizer que um sinal FM é representado por $$s_{fm} = \sin(2\pi f_0t + \beta\sin(2\pi f_m t))$$
onde $f_0$ é a frequencia de portadora, $f_m$ é a frequência do sinal de mensagem e $\beta$ é o índice de modulação (para mais informações ver [aqui](https://www.rfcafe.com/references/electrical/frequency-modulation.htm) ou [aqui](http://paginapessoal.utfpr.edu.br/alessandro/disciplinas-do-semestre/principios-de-comunicacao/aulas/segunda-prova/Modulacao%20Angular.pdf)).

Crie um sinal FM e proceda com a visualização do espectrograma desse sinal. Você deve visualizar uma senoide. Como deve ser a visualização de um sinal AM (explique) ?